# Inteprate Clusters
ref:
- 

# Set up

In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle

## Load Model
This notebook assumed that there is a model saved locally.

In [ ]:
# load the model
model_filepath = '../../.aml/models/mini-batch-k-means-customer-segmentation.pkl'
km = pickle.load(open(model_filepath, "rb"))

# use model to predict
#y = km.predict(X_mixed)

## Load `PowerTransformer`
This notebook assumed that there is a `PowerTransformer` saved locally.

In [ ]:
ptransformer_filepath = f'../../.aml/models/powertransformer.pkl'
ptransformer = pickle.load(open(ptransformer_filepath, "rb"))

### Get Cluster Centroid

#### Inverse Transformed Cluster Centroid for Interpretation

In [ ]:
column_names=['Recency(Days)',	'Frequency', 'Monetary(£)']

km_cluster_centers = pd.DataFrame(ptransformer.inverse_transform(km.cluster_centers_), columns=column_names)
km_cluster_centers['Cluster_Label'] = km_cluster_centers.index
km_cluster_centers

#### Normalised Cluster Centroid for Visualisation

In [ ]:
column_names=['Recency(Days)',	'Frequency', 'Monetary(£)']

km_cluster_centers_transformed = pd.DataFrame(km.cluster_centers_, columns=column_names)
km_cluster_centers_transformed['Cluster_Label'] = km_cluster_centers_transformed.index
km_cluster_centers_transformed

### Visualisation

In [ ]:
# Reshape data for snake plot
cluster_melt = pd.melt(km_cluster_centers_transformed.reset_index(),
                       id_vars = ['Cluster_Label'],
                       value_vars = column_names,
                       var_name = 'Features',
                       value_name = 'Value')

In [ ]:
def snakeplot(data):
    _ = plt.figure(figsize=(10,6))
    _ = plt.title('Snake plot of features for each cluster')
    _ = sns.lineplot(x="Features", y="Value", hue='Cluster_Label', data=data, palette='Set2')

In [ ]:
snakeplot(cluster_melt)

### Possible Intepretation

Cluster | Recency(Days)                 | Frequency (last 6 months) | Monetary(£) last 6 months | Recommended Action / Thoughts |
---     | ---                           | ---                       | ---                       | --- |
0       | Active since last 2 weeks     | 100 transactions          | 1500                      | Very frequent customers who rely on online shopping. |
1       | Active since last 3 months    | 30 transactions           | 500                       | Inactive for a while, but has been typical in spending. |
2       | Active since last 3 months    | 7 transactions            | 150                       | Inactive for a while, and probably not rugular customer. |
3       | Active since last 2 weeks     | 20 transactions           | 350                       | Active and regular, typical spending. |
